# Ch05. Crawling with API

## 1. Naver 검색-Blog 

In [3]:
import os
import sys
import urllib.request
client_id = "Qcbdc1pLcZmksJCNWbMH"
client_secret = "DD4dbeoPmG"
encText = urllib.parse.quote("빅데이터")
url = "https://openapi.naver.com/v1/search/blog?query=" + encText # json 결과
# url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # xml 결과
request = urllib.request.Request(url)
request.add_header("X-Naver-Client-Id",client_id)
request.add_header("X-Naver-Client-Secret",client_secret)
response = urllib.request.urlopen(request)
rescode = response.getcode()
if(rescode==200):
    response_body = response.read()
    print(response_body.decode('utf-8'))
else:
    print("Error Code:" + rescode)

{
"lastBuildDate": "Sun, 21 Mar 2021 20:29:48 +0900",
"total": 803885,
"start": 1,
"display": 10,
"items": [
{
"title": "세라마이드 <b>빅데이터</b> 수집가의 최종섭취법",
"link": "https:\/\/blog.naver.com\/nana6564?Redirect=Log&logNo=222256030177",
"description": "않는 <b>빅데이터</b>를 수집하다 알게 된 게 세라마이드라는 성분이었네요 이건 우리 피부를 구성하는... 않은 <b>빅데이터</b>를 수집하면서 알게 된 최종섭취법을 함께 공유해드렸는데 모두가 피부 고민 없는 그날까지 함께... ",
"bloggername": "은강 천연비누",
"bloggerlink": "https://blog.naver.com/nana6564",
"postdate": "20210225"

},
{
"title": "운동없이 다이어트 <b>빅데이터</b> 수집가의 200% 찐 후기!",
"link": "https:\/\/blog.naver.com\/dynamickimi?Redirect=Log&logNo=222247978609",
"description": "<b>빅데이터</b> 수집가 답게 여기저기 검색하며 차곡차곡 내공이 쌓였는데 경험자 입장으로 정말 찐! 이라고... 끝에 <b>빅데이터</b> 수집가의 결론을 적어 보자면! 하나를 먹더라도 고유한 당 지수를 꼭 확인하셔야 해요.... ",
"bloggername": "LOVE and LOVE",
"bloggerlink": "https://blog.naver.com/dynamickimi",
"postdate": "20210218"

},
{
"title": "<b>빅데이터</b>학원 수강생들의 찐 후기!",
"link": "https:\/\/blog.naver.com\/romeyo1013?Redirect=Log&logNo=222277541086",
"de

### (참고) JSON
####   Python (tuple, list) -->json (array) : json.dumps
####  json (array) --> Python (list) : json.loads

In [21]:
import json
student_data = {
    "1.FirstName": "Gildong",
    "2.LastName": "Hong",
    "3.Age": 20, 
    "4.University": "Yonsei University",
    "5.Courses": [
        {
            "Major": "Statistics", 
            "Classes": ["Probability", 
                        "Generalized Linear Model", 
                        "Categorical Data Analysis"]
        }, 
        {
            "Minor": "ComputerScience", 
            "Classes": ["Data Structure", 
                        "Programming", 
                        "Algorithms"]
        }
    ]
} 
st_json = json.dumps(student_data,indent=4,sort_keys=True)

print(st_json) # JSON 형식의 문자열 출력

{
    "1.FirstName": "Gildong",
    "2.LastName": "Hong",
    "3.Age": 20,
    "4.University": "Yonsei University",
    "5.Courses": [
        {
            "Classes": [
                "Probability",
                "Generalized Linear Model",
                "Categorical Data Analysis"
            ],
            "Major": "Statistics"
        },
        {
            "Classes": [
                "Data Structure",
                "Programming",
                "Algorithms"
            ],
            "Minor": "ComputerScience"
        }
    ]
}


In [22]:
# Make JSON string objects
st_python = json.loads(st_json)
print(st_python)

{'1.FirstName': 'Gildong', '2.LastName': 'Hong', '3.Age': 20, '4.University': 'Yonsei University', '5.Courses': [{'Classes': ['Probability', 'Generalized Linear Model', 'Categorical Data Analysis'], 'Major': 'Statistics'}, {'Classes': ['Data Structure', 'Programming', 'Algorithms'], 'Minor': 'ComputerScience'}]}


#### reponse 의 내용 보기

In [23]:
import json
dict = json.loads(response_body.decode('utf-8'))
print(dict['lastBuildDate'])

Sun, 21 Mar 2021 20:29:48 +0900


### 전체프로그램

In [50]:
import os
import sys
import urllib.request
import datetime
import time
import json
client_id = "Qcbdc1pLcZmksJCNWbMH"
client_secret = "DD4dbeoPmG"

#[CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)
    # 참고: 네이버 API가이드에 따르면, start = 1000까지만 허용하므로, 그 이상할 경우 에러발생 가능 
    
    url = base + node + parameters
    responseDecode = getRequestUrl(url) #[CODE 1]

    if (responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

#[CODE 3]
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')
    jsonResult.append({'cnt':cnt, 'title':title, 'description': description,'org_link':org_link, 'link': org_link, 'pDate':pDate})
    return

#[CODE 0]
def main():
    node = 'news' #크롤링할 대상
    srcText = input('검색어를 입력하세요:')
    cnt = 0
    jsonResult = []
    jsonResponse = getNaverSearch(node, srcText, 1, 100) #[CODE 2]
    total = jsonResponse['total']
    while ((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt += 1
            getPostData(post, jsonResult, cnt) #[CODE 3]

        start = jsonResponse['start'] + jsonResponse['display']
        print("start:", start)
        jsonResponse = getNaverSearch(node, srcText, start, 100) #[CODE 2]

    print('전체 검색 : %d 건' %total)

    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True, ensure_ascii = False)
        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))

if __name__ == '__main__':
    main()


검색어를 입력하세요:Coner Stone
[2021-03-21 22:24:24.592612] Url Request Success
start: 6
[2021-03-21 22:24:24.753182] Url Request Success
전체 검색 : 5 건
가져온 데이터 : 5 건
Coner Stone_naver_news.json SAVED


## 2. 공공데이터 API기반 크롤링

In [3]:
# data.go.kr의 Python 샘플 코드 : 
# urllib 의 버전업 변화에 따른 변황 반영함.


from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
# queryParams = '?' + urlencode({ quote_plus('ServiceKey') : '인증키, quote_plus('YM') : '201201', quote_plus('NAT_CD') : '112', quote_plus('ED_CD') : 'D' })
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : 'vwjVSLvyj+hL/qV8bvlVNK3HD50j+hBZCA6ODLobFoqroQHYPseuAsdY0ouQqfTA4piBuKjSdauycXXfLxCl3A==', quote_plus('YM') : '201201', quote_plus('NAT_CD') : '112', quote_plus('ED_CD') : 'D' })

req = Request(url + queryParams)
req.get_method = lambda: 'GET'
response_body = urlopen(req).read()
print(response_body)

b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>0000</resultCode><resultMsg>OK</resultMsg></header><body><items/><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>'


In [7]:
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus

url = 'http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList'
sk = 'vwjVSLvyj+hL/qV8bvlVNK3HD50j+hBZCA6ODLobFoqroQHYPseuAsdY0ouQqfTA4piBuKjSdauycXXfLxCl3A=='
#sk = 'vwjVSLvyj%252BhL%252FqV8bvlVNK3HD50j%252BhBZCA6ODLobFoqroQHYPseuAsdY0ouQqfTA4piBuKjSdauycXXfLxCl3A%253D%253D'
#sk = 'vwjVSLvyj%2BhL%2FqV8bvlVNK3HD50j%2BhBZCA6ODLobFoqroQHYPseuAsdY0ouQqfTA4piBuKjSdauycXXfLxCl3A%3D%3D'
queryParams = '?' + urlencode({ quote_plus('ServiceKey') : sk, quote_plus('YM') : '201201', quote_plus('NAT_CD') : '112', quote_plus('ED_CD') : 'D' })
print(queryParams)
request = Request(url + queryParams)
request.get_method = lambda: 'GET'
response_body = urlopen(request).read()
print(response_body )

?ServiceKey=vwjVSLvyj%2BhL%2FqV8bvlVNK3HD50j%2BhBZCA6ODLobFoqroQHYPseuAsdY0ouQqfTA4piBuKjSdauycXXfLxCl3A%3D%3D&YM=201201&NAT_CD=112&ED_CD=D
b'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>0000</resultCode><resultMsg>OK</resultMsg></header><body><items/><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>0</totalCount></body></response>'


### 교재 전체 코드

In [35]:
import os
import sys
import urllib.request
import datetime
import time
import json
import pandas as pd
import xml.etree.ElementTree as ET

#ServiceKey = 'vwjVSLvyj%2BhL%2FqV8bvlVNK3HD50j%2BhBZCA6ODLobFoqroQHYPseuAsdY0ouQqfTA4piBuKjSdauycXXfLxCl3A%3D%3D'
#vwjVSLvyj%2BhL%2FqV8bvlVNK3HD50j%2BhBZCA6ODLobFoqroQHYPseuAsdY0ouQqfTA4piBuKjSdauycXXfLxCl3A%3D%3D
sk = 'vwjVSLvyj+hL/qV8bvlVNK3HD50j+hBZCA6ODLobFoqroQHYPseuAsdY0ouQqfTA4piBuKjSdauycXXfLxCl3A=='

#[CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s] Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e:
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
def getTourismStatsItem(yyyymm, national_code, ed_cd):
    service_url = "http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList"

#    parameters = "?_type=json&ServiceKey=" + ServiceKey #인증키
#    parameters += "&YM=" + yyyymm
#    parameters += "&NAT_CD=" + national_code
#    parameters += "&ED_CD=" + ed_cd

    parameters = '?' +  urlencode({ quote_plus('ServiceKey'):sk,quote_plus('YM'): yyyymm,
                                    quote_plus('NAT_CD'):national_code, quote_plus('ED_CD'):ed_cd})                                    
    url = service_url + parameters
    print(url)
    retData = getRequestUrl(url) #[CODE 1]
    if (retData == None):
        return None
    else:
        return retData

#[CODE 3]
def getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear):
    jsonResult = []
    result = []
    for year in range(nStartYear, nEndYear+1):
        for month in range(1, 13):
            xmlData = None
            yyyymm = "{0}{1:0>2}".format(str(year), str(month))
            #jsonData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) #[CODE 2]
            xmlData = getTourismStatsItem(yyyymm, nat_cd, ed_cd) #[CODE 2]
            print("getTourismStatsService- (1) xmlData", xmlData)
            if xmlData == None:
                dataEND = "{0}{1:0>2}".format(str(year), str(month-1))
                print("데이터 없음.... \n 제공되는 통계 데이터는 %s년 %s월까지입니다." 
                      %(str(year), str(month-1)))
                break
            else:
                root = ET.fromstring(xmlData)            
            for item in root.iter('item'): 
                natName = item.find('natKorNm').text
                natName = natName.replace(' ', '')
                num = item.find('num').text
                ed = item.find('ed').text
            print('[ %s_%s : %s ]' %(natName, yyyymm, num))
            print('-----------------------------------------------------')
            jsonResult.append({'nat_name': natName, 'nat_cd': nat_cd,'yyyymm': yyyymm, 'visit_cnt': num})
            result.append([natName, nat_cd, yyyymm, num])
        if xmlData == None:
                break
    return (jsonResult, result, natName, ed, dataEND)

#[CODE 0]
def main():
    jsonResult = []
    result = []

    print("<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>")
    nat_cd = input('국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : ')
    nStartYear = int(input('데이터를 몇 년부터 수집할까요? : '))
    nEndYear = int(input('데이터를 몇 년까지 수집할까요? : '))
    ed_cd = "E" #E : 방한외래관광객, D : 해외 출국
    jsonResult, result, natName, ed, dataEND = getTourismStatsService(nat_cd, ed_cd, nStartYear, nEndYear) 

    #파일저장 1 : json 파일
    with open('./%s_%s_%d_%s.json' % (natName, ed, nStartYear, dataEND),
                 'w', encoding = 'utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent = 4, sort_keys = True,
                                     ensure_ascii = False)
        outfile.write(jsonFile)
    #파일저장 2 : csv 파일
    columns = ["입국자국가", "국가코드", "입국연월", "입국자 수"]
    result_df = pd.DataFrame(result, columns = columns)
    result_df.to_csv('./%s_%s_%d_%s.csv' % (natName, ed, nStartYear,
                        dataEND), index=False, encoding='cp949')

if __name__ == '__main__':
    main()
                  

<< 국내 입국한 외국인의 통계 데이터를 수집합니다. >>
국가 코드를 입력하세요(중국: 112 / 일본: 130 / 미국: 275) : 130
데이터를 몇 년부터 수집할까요? : 2017
데이터를 몇 년까지 수집할까요? : 2018
http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?ServiceKey=vwjVSLvyj%2BhL%2FqV8bvlVNK3HD50j%2BhBZCA6ODLobFoqroQHYPseuAsdY0ouQqfTA4piBuKjSdauycXXfLxCl3A%3D%3D&YM=201701&NAT_CD=130&ED_CD=E
[2021-03-24 16:03:43.115283] Url Request Success
getTourismStatsService- (1) xmlData <?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>0000</resultCode><resultMsg>OK</resultMsg></header><body><items><item><ed>방한외래관광객</ed><edCd>E</edCd><natCd>130</natCd><natKorNm>일  본</natKorNm><num>154862</num><rnum>1</rnum><ym>201701</ym></item></items><numOfRows>10</numOfRows><pageNo>1</pageNo><totalCount>1</totalCount></body></response>
[ 일본_201701 : 154862 ]
-----------------------------------------------------
http://openapi.tour.go.kr/openapi/service/EdrcntTourismStatsService/getEdrcntTourismStatsList?Ser